# **DIMEMEX — Complete Project Pipeline Notebook**

*Multilingual Meme Translation & Hate Speech Analysis*

---

## **Members**

* **Bárbara** (Text)
* **Amanda** (Text + Description)
* **Juan David Nieto** (Text + Description + Image)
* **Luisa** (Image)

---

## **Project Overview**

This project analyzes whether **offensive or hate speech content in memes is preserved after translating the memes from Spanish to Portuguese**.

We work with the **DIMEMEX** dataset, which contains:

* Meme **text**
* Meme **description**
* Meme **image**
* Labels: *hate speech*, *inappropriate content*, *neither*

This project has **two main tasks**:

### **Task 1 — Translation Quality Evaluation**

Translate the Spanish text to Portuguese and evaluate translation quality using standard NLP metrics:

* **BLEURT**
* **BERTScore**
* **COMET-Kiwi**
* **chrF**

### **Task 2 — Hate Speech Detection (Multimodal Fine-Tuning)**

Fine-tune models to classify:

* Hate speech
* Inappropriate content
* Neither

We fine-tune under four input settings:

1. Text
2. Text + Description
3. Text + Description + Image (Multimodal)
4. Image

---

## **📌 Objectives**

1. Evaluate whether offensive content is **maintained or lost** during translation.
2. Compare performance of **original Spanish memes vs. translated Portuguese memes**.
3. Train and evaluate multimodal detectors to classify hate speech.
4. Analyze cases where the label changes across languages.
5. Perform **human qualitative analysis** on inconsistent cases.



# Imports

# 🛠️ 1. Install Dependencies

In [ ]:
# Para ver o progresso, remova o -q
!pip install accelerate datasets peft bitsandbytes tensorboard pandas
!pip install flash-attn --no-build-isolation 

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [160 lines of output]
      
      
      torch.__version__  = 2.9.1+cu128
      
      
      /usr/lib/python3/dist-packages/setuptools/installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
      running bdist_wheel
      Guessing wheel URL:  https://github.com/Dao-AILab/flash-attention/releases/download/v2.8.3/flash_attn-2.8.3+cu12torch2.9cxx11abiTRUE-cp310-cp310-linux_x86_64.whl
      Precompiled wheel not found. Building from source...
      W1120 13:26:19.160000 232005 torch/utils/cpp_extension.py:630] Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
      running build
      running build_py
      creating build
      creating build/lib.linux-x86_64-3.10
      creating b

In [ ]:
import torch
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from transformers import AutoProcessor, BitsAndBytesConfig, Idefics3ForConditionalGeneration, TrainingArguments, Trainer
from PIL import Image
import pandas as pd
import os
from datasets import Dataset

In [ ]:
USE_LORA = False
USE_QLORA = True

model_id = "HuggingFaceTB/SmolVLM-256M-Instruct" 

# 🗂️ 2. Upload Data

In [ ]:
# === Caminhos dos CSVs (ajuste se preferir) ===
CSV_TRAIN = "../train/dados_espanhol_balanceado.csv"
CSV_VAL   = "../validation/dados_espanhol.csv"
CSV_TEST  = "../test/dados_espanhol.csv"
# === Caminhos das pastas de imagens ===
TRAIN_IMAGES_DIR = "train_images"
VAL_IMAGES_DIR   = "validation_images"
TEST_IMAGES_DIR  = "test_images"
# Carregar CSVs
df_train = pd.read_csv(CSV_TRAIN)
df_val   = pd.read_csv(CSV_VAL)
df_test  = pd.read_csv(CSV_TEST)

print("CSV train:", df_train.shape)
print("CSV val:", df_val.shape)
print("CSV test:", df_test.shape)


# ⚙️ 3. Main Configurations

In [ ]:
# --- Model & Training Params ---
BATCH_SIZE = 2
EPOCHS = 4
LR = 2e-5
OUTPUT_DIR = "./SmolVLM_DIMEMEX_ImageOnly"
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Label distribution (string labels only) ---
print("\n📊 Quantidade de exemplos por label:")
print(df_train["label"].value_counts())


📊 Quantidade de exemplos por label:
label
hate speech              600
inappropriate content    472
neither                  386
Name: count, dtype: int64


# 📦 4. Prepare Train, Validation, and Test Datasets

In [ ]:
# Os dados já estão divididos

df_train["image_path"] = df_train["image_path"].apply(lambda x: os.path.join(TRAIN_IMAGES_DIR, x))
df_val["image_path"]   = df_val["image_path"].apply(lambda x: os.path.join(VAL_IMAGES_DIR, x))
df_test["image_path"]  = df_test["image_path"].apply(lambda x: os.path.join(TEST_IMAGES_DIR, x))

ds_train = Dataset.from_pandas(df_train.reset_index(drop=True))
ds_val = Dataset.from_pandas(df_val.reset_index(drop=True))
ds_test = Dataset.from_pandas(df_test.reset_index(drop=True))

print(f"Train: {len(ds_train)} | Validation: {len(ds_val)} | Test: {len(ds_test)}")

Train: 1458 | Validation: 322 | Test: 648


# 🧩 5. Processor and Pre-processing **(Image)**

In [ ]:
processor = AutoProcessor.from_pretrained(
    model_id
)

if USE_QLORA or USE_LORA:
    lora_config = LoraConfig(
        r=8,
        lora_alpha=8,
        lora_dropout=0.1,
        target_modules=['down_proj','o_proj','k_proj','q_proj','gate_proj','up_proj','v_proj'],
        use_dora=False if USE_QLORA else True,
        init_lora_weights="gaussian"
    )
    lora_config.inference_mode = False
    if USE_QLORA:
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.bfloat16
        )

    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_id,
        quantization_config=bnb_config if USE_QLORA else None,
        _attn_implementation="flash_attention_2",
        device_map="auto"
    )
    model.add_adapter(lora_config)
    model.enable_adapters()
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, lora_config)
    print(model.get_nb_trainable_parameters())
else:
    model = Idefics3ForConditionalGeneration.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        _attn_implementation="flash_attention_2",
    ).to(device)

    for param in model.model.vision_model.parameters():
        param.requires_grad = False

✅ Datasets recriados com sucesso!
Colunas: ['images', 'messages', 'image_path']
Exemplo de formatação:
User: Clasifica este meme: hate speech, inappropriate content, o neither
Assistant: neither


# 🧮 7. Training Configuration

In [ ]:
image_token_id = processor.tokenizer.additional_special_tokens_ids[
            processor.tokenizer.additional_special_tokens.index("<image>")]

def format_with_original_label(example, label):
    path = example["image_path"]
    try:
        if not os.path.exists(path):
            image = Image.new("RGB", (224, 224), "black")
        else:
            image = Image.open(path).convert("RGB")
            image.thumbnail((512, 512))  
    except Exception:
        image = Image.new("RGB", (224, 224), "black")

    return {"image": image, "label": label, "image_path": path}

# Criar datasets
train_data = [
    format_with_original_label({"image_path": row["image_path"]}, row["label"])
    for _, row in df_train.iterrows()
]

val_data = [
    format_with_original_label({"image_path": row["image_path"]}, row["label"])
    for _, row in df_val.iterrows()
]

ds_train = Dataset.from_list(train_data)
ds_val = Dataset.from_list(val_data)

def collate_fn(examples):
    texts = []
    images = []
    for example in examples:
        image = example["image"]
        if image.mode != 'RGB':
            image = image.convert('RGB')
        
        label = example["label"]
        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Clasifica este meme: hate speech, inappropriate content, o neither"}
                ]
            },
            {
                "role": "assistant",
                "content": [
                    {"type": "text", "text": label}
                ]
            }
        ]
        text = processor.apply_chat_template(messages, add_generation_prompt=False)
        texts.append(text.strip())
        images.append([image])

    batch = processor(text=texts, images=images, return_tensors="pt", padding=True)
    labels = batch["input_ids"].clone()
    labels[labels == processor.tokenizer.pad_token_id] = -100
    labels[labels == image_token_id] = -100
    batch["labels"] = labels

    return batch

data_collator = collate_fn
# Instanciar o data collator

In [ ]:
# Configurar Argumentos de Treino
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=2,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    fp16=False,
    bf16=True,
    tf32=True,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    remove_unused_columns=False,
    report_to="none",
    dataloader_num_workers=4,
    ddp_find_unused_parameters=False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    data_collator=data_collator,
)

# 9. Train Model

In [ ]:
trainer.train()
print("Fine-tuning complete.")

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/luisastellet/.local/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/luisastellet/.local/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 54, in fetch
    return self.collate_fn(data)
  File "/tmp/ipykernel_228903/1094426231.py", line 16, in __call__
    batch = self.processor(text=texts, images=images, return_tensors="pt", padding=True)
  File "/tmp/ipykernel_228903/3661737659.py", line 26, in __call__
    text_inputs = self.tokenizer(
TypeError: GPT2TokenizerFast(name_or_path='HuggingFaceTB/SmolVLM-256M-Instruct', vocab_size=49152, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='left', special_tokens={'bos_token': '<|im_start|>', 'eos_token': '<end_of_utterance>', 'unk_token': '<|endoftext|>', 'pad_token': '<|im_end|>', 'additional_sp

# 10. Loss Curves

In [ ]:
# Garante que o diretório de saída existe
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Acessa o histórico direto da memória
logs = trainer.state.log_history

train_steps = [x["step"] for x in logs if "loss" in x]
train_loss = [x["loss"] for x in logs if "loss" in x]

eval_steps = [x["step"] for x in logs if "eval_loss" in x]
eval_loss = [x["eval_loss"] for x in logs if "eval_loss" in x]

plt.figure(figsize=(10, 5))
plt.plot(train_steps, train_loss, label="Train Loss")
plt.plot(eval_steps, eval_loss, label="Eval Loss", marker='o')
plt.title("Training and Evaluation Loss")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)

# --- SALVAR O GRÁFICO ---
plot_path = os.path.join(OUTPUT_DIR, "loss_curve.png")
plt.savefig(plot_path)
print(f"📉 Gráfico de Loss salvo em: {plot_path}")

plt.show()

### **Só pode rodar a célula abaixo quando estiver satisfeito com os resultado! Parão não ter vazamento de dados**

# ✅ 11. Final Evaluation and Metrics Summary

In [ ]:
# Use o modelo treinado
model = trainer.model
model.eval()

test_predictions = []
test_ground_truth = []
valid_labels = ["hate speech", "inappropriate content", "neither"]

print("🚀 Rodando avaliação final no conjunto de teste...")

# Loop pelo Test Set
for example in tqdm(ds_test):
    try:
        image = Image.open(example['image_path']).convert("RGB")
    except:
        continue

    prompt_text = """"
                Analiza esta imagen de meme y clasifícala en una de estas tres categorías: 
                - hate speech
                - inappropriate content
                - neither
                ¿Cuál es la categoría correcta para este meme?
                """

    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image"},
                {"type": "text", "text": prompt_text}
            ]
        }
    ]

    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=image, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        generated_ids = model.generate(**inputs, max_new_tokens=20, do_sample=False)

    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Limpeza da resposta
    if "Assistant:" in generated_text:
        prediction = generated_text.split("Assistant:")[-1].strip()
    elif "assistant" in generated_text:
         prediction = generated_text.split("assistant")[-1].strip()
    else:
        prediction = generated_text.strip()

    test_predictions.append(prediction)
    test_ground_truth.append(example['label'])

In [ ]:
# --- Processamento e Salvamento dos Resultados ---

# 1. Limpeza e Filtro
cleaned_preds = []
for p in test_predictions:
    p_clean = p.lower().replace(".", "").strip()
    if p_clean in valid_labels:
        cleaned_preds.append(p_clean)
    else:
        cleaned_preds.append("unknown") # Marca erro de geração

# 2. Preparar listas finais
final_preds = []
final_gt = []
for p, g in zip(cleaned_preds, test_ground_truth):
    if p in valid_labels:
        final_preds.append(p)
        final_gt.append(g)

# 3. SALVAR PREDIÇÕES (CSV)
# Isso cria uma tabela com: Label Real | Predição | Acertou?
df_results = pd.DataFrame({
    "ground_truth": final_gt,
    "prediction": final_preds
})
df_results["correct"] = df_results["ground_truth"] == df_results["prediction"]
csv_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
df_results.to_csv(csv_path, index=False)
print(f"\n💾 Tabela de predições salva em: {csv_path}")

# 4. Calcular Métricas
report_str = classification_report(final_gt, final_preds, labels=valid_labels)
results = {
    "accuracy": accuracy_score(final_gt, final_preds),
    "f1_weighted": f1_score(final_gt, final_preds, average="weighted"),
    "precision_weighted": precision_score(final_gt, final_preds, average="weighted", zero_division=0),
    "recall_weighted": recall_score(final_gt, final_preds, average="weighted", zero_division=0)
}

# 5. SALVAR RELATÓRIO (TXT e JSON)
txt_path = os.path.join(OUTPUT_DIR, "classification_report.txt")
with open(txt_path, "w") as f:
    f.write(report_str)
    f.write("\n\nSummary Metrics:\n")
    f.write(json.dumps(results, indent=4))

json_path = os.path.join(OUTPUT_DIR, "metrics.json")
with open(json_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"📄 Relatórios salvos em: {txt_path} e {json_path}")

# Exibir na tela
print(f"\n{len(final_preds)} / {len(test_predictions)} predições válidas.")
print("\n📊 Resultados Finais:")
print(report_str)

# 💾 12. Save Final Model

In [ ]:
# Save the LoRA adapters
model.save_pretrained(OUTPUT_DIR)
# Save the processor
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Fine-tuning complete. Model saved to: {OUTPUT_DIR}")